In [1]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [4]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-02-17 21:10:22,447 INFO: Initializing external client
2025-02-17 21:10:22,448 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-17 21:10:23,222 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213634
Fetching data from 2025-01-20 02:10:22.445130+00:00 to 2025-02-18 01:10:22.445130+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.44s) 


In [5]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-02-17 21:10:36,982 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-17 21:10:36,988 INFO: Initializing external client
2025-02-17 21:10:36,989 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-17 21:10:37,716 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213634

In [6]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

2025-02-17 21:10:39,616 WARNING: DeprecationWarning: The LGBMRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.



In [7]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,263,119.0,2025-02-18 03:00:00+00:00
1,132,26.0,2025-02-18 03:00:00+00:00
2,147,0.0,2025-02-18 03:00:00+00:00
3,13,23.0,2025-02-18 03:00:00+00:00
4,155,0.0,2025-02-18 03:00:00+00:00
...,...,...,...
248,236,124.0,2025-02-18 03:00:00+00:00
249,174,0.0,2025-02-18 03:00:00+00:00
250,167,0.0,2025-02-18 03:00:00+00:00
251,246,100.0,2025-02-18 03:00:00+00:00


In [8]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-02-17 21:10:39,833 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-17 21:10:39,840 INFO: Initializing external client
2025-02-17 21:10:39,840 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-17 21:10:40,616 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213634


AttributeError: module 'src.config' has no attribute 'FEATURE_GROUP_MODEL_PREDICTION'

In [9]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

NameError: name 'feature_group' is not defined